# Part one: Data importing and Cleaning
This is a quick implementation of Elastic Searching using the Harry Potter where this file imports and cleans the Data to work with in Elastic

The dataset we will use in this project is a Harry Potter characters table. This has been created on Kaggle, see original link here..
https://www.kaggle.com/datasets/gulsahdemiryurek/harry-potter-dataset

In [5]:
import pandas as pd
import re
hp_data = pd.read_csv("Data/Characters.csv", sep = ";" )
hp_data.head()

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1,Harry James Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,31 July 1980,NaN
1,2,Ronald Bilius Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1 March 1980,NaN
2,3,Hermione Jean Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,"19 September, 1979",NaN
3,4,Albus Percival Wulfric Brian Dumbledore,Male,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,Human,Half-blood,Silver| formerly auburn,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,Late August 1881,"30 June, 1997"
4,5,Rubeus Hagrid,Male,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",NaN,Half-Human/Half-Giant,Part-Human (Half-giant),Black,Black,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,6 December 1928,NaN


### Data Preprocessing and cleaning
After utlising gpt for inspection, I will make the following changes

**Handle multi-line values:** Some cells in the CSV file contain newline characters (\n). You can either replace these newlines with a space or keep them as-is, depending on your preference.

**Handle missing values:** Some cells in the CSV file are empty. You can either replace these empty values with a specific string (e.g., "N/A") or leave them as empty strings, depending on your needs.

**Handle data types:** Elasticsearch expects specific data types for different fields. You may need to convert some fields to their appropriate data types, such as converting numeric values to integers or floats, and date values to a proper date format.

**Handle special characters:** Depending on your data, you may need to handle special characters or encode the data in a specific format (e.g., UTF-8) to ensure proper indexing and searching in Elasticsearch.

In [8]:
hp_data = hp_data.fillna("Unknown")
df_obj = hp_data.select_dtypes(["object"])
hp_data = df_obj.map(lambda x: re.sub(r'[^ \w+]', '', str(x)))
hp_data["Birth"]= hp_data["Birth"].apply(lambda x: pd.to_datetime(x, errors="coerce"))
hp_data["Death"]= hp_data["Death"].apply(lambda x: pd.to_datetime(x, errors="coerce"))
hp_data = hp_data.fillna("Unknown")
hp_data["Blood status"].unique()
hp_data["Blood status"] = hp_data["Blood status"].str.lower()
hp_data["Blood status"] = hp_data["Blood status"].replace(to_replace=["halfbloodorpureblood", "purebloodorhalfblood"], value = "pureblood or halfblood")
hp_data.head()

,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,Harry James Potter,Male,Student,Gryffindor,11 Holly phoenix feather,Stag,Human,halfblood,Black,Bright green,Albus Dumbledore Dumbledores Army Order of t...,Parseltongue Defence Against the Dark Arts Se...,1980-07-31 00:00:00,Unknown
1,Ronald Bilius Weasley,Male,Student,Gryffindor,12 Ash unicorn tail hair,Jack Russell terrier,Human,pureblood,Red,Blue,Dumbledores Army Order of the Phoenix Hogwar...,Wizard chess Quidditch goalkeeping,1980-03-01 00:00:00,Unknown
2,Hermione Jean Granger,Female,Student,Gryffindor,10¾ vine wood dragon heartstring,Otter,Human,muggleborn,Brown,Brown,Dumbledores Army Order of the Phoenix Hogwar...,Almost everything,1979-09-19 00:00:00,Unknown
3,Albus Percival Wulfric Brian Dumbledore,Male,Headmaster,Gryffindor,15 Elder Thestral tail hair core,Phoenix,Human,halfblood,Silver formerly auburn,Blue,Dumbledores Army Order of the Phoenix Hogwar...,Considered by many to be one of the most power...,Unknown,1997-06-30 00:00:00
4,Rubeus Hagrid,Male,Keeper of Keys and Grounds Professor of Care ...,Gryffindor,16 Oak unknown core,Unknown,HalfHumanHalfGiant,parthumanhalfgiant,Black,Black,Albus Dumbledore Order of the Phoenix Hogwar...,Resistant to stunning spells above average str...,1928-12-06 00:00:00,Unknown


Save our clean data for future use

In [9]:
hp_data.to_csv("data/clean_characters.csv")

### Import our cleaned data

In [10]:

hp_data = pd.read_csv("Data/clean_characters.csv")